## Imports

In [2]:
import re
import time
import pandas as pd
import warnings
from anthropic import Anthropic
from pathlib import Path
from tqdm import tqdm

## Configs

In [4]:
anthropic = Anthropic(api_key=open('../token.txt','r').read())
api_key= open('../token.txt','r').read()
warnings.filterwarnings('ignore')
raw_data_path = Path('../00_dados/raw/sample_teste_nlp.csv')

## Testando API

In [6]:
'''# Fazer uma chamada simples
message = anthropic.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    messages=[{
        "role": "user",
        "content": "Olá, como vai?"
    }]
)

print(message.content)'''

'# Fazer uma chamada simples\nmessage = anthropic.messages.create(\n    model="claude-3-sonnet-20240229",\n    max_tokens=1000,\n    messages=[{\n        "role": "user",\n        "content": "Olá, como vai?"\n    }]\n)\n\nprint(message.content)'

## Funções Auxiliares

In [39]:
def limpar_texto_com_claude(df, coluna_texto, api_key):
    """
    Função para limpar textos de um DataFrame usando a API do Claude,
    com barra de progresso.
    
    Parâmetros:
    df (pandas.DataFrame): DataFrame com a coluna de texto
    coluna_texto (str): Nome da coluna que contém os textos
    api_key (str): Chave da API da Anthropic
    
    Retorna:
    pandas.DataFrame: DataFrame com a nova coluna de texto limpo
    """
    # Inicializar o cliente Claude
    anthropic = Anthropic(api_key=api_key)
    
    def processar_texto(texto):
        try:
            # Adicionar pequeno delay para evitar limite de requisições
            time.sleep(0.5)
            
            # Instrução para o Claude sobre como limpar o texto
            prompt = f"""
            Por favor, limpe o seguinte texto removendo:
            - Remova todas as tags html
            - Remova toda simbologia rft
            - Simbologia de quebra de linhas e tabulação
            - Caracteres especiais desnecessários
            - Espaços extras
            - Quebras de linha desnecessárias
            - Formatação inconsistente
            - Deixe tudo em letras minúsculas
            
            Mantenha o significado original do texto e um texto que faça sentido com palavras reais.
            
            Texto original: {texto}
            
            Retorne apenas o texto limpo, sem explicações adicionais.
            """
            
            # Fazer a chamada à API
            response = anthropic.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=1000,
                messages=[{
                    "role": "user",
                    "content": prompt
                }]
            )
            
            return response.content
        except Exception as e:
            print(f"\nErro ao processar texto: {e}")
            return texto  # Retorna o texto original em caso de erro
    
    # Lista para armazenar os resultados
    textos_limpos = []
    
    # Criar barra de progresso
    print(f"Processando {len(df)} textos...")
    for texto in tqdm(df[coluna_texto], desc="Limpando textos", unit="texto"):
        texto_limpo = processar_texto(texto)
        texto_limpo = str(texto_limpo[0])
        textos_limpos.append(texto_limpo[16:-15])
    
    # Adicionar coluna com os textos limpos
    df['texto_limpo'] = textos_limpos
    
    return df

## Importação dos dados

In [10]:
df = pd.read_csv(raw_data_path)
df.head()

,patient_id,age,sex,ehr_date,text
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:..."
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:..."
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...


## Processamento dos dados

In [12]:
df_teste = df.iloc[0:5]
df_limpo = limpar_texto_com_claude(df_teste, 'text', api_key)

Processando 5 textos...


Limpando textos: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:36<00:00,  7.34s/texto]


In [13]:
df_limpo

,patient_id,age,sex,ehr_date,text,texto_limpo
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:...",anamnese - centro clínico queixa principal: do...
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:...","paciente com cefaleia occipital, crise há 3 di..."
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,07:00 recebo plantão em posto de enfermagem.\n...
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D,calcio vit d
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,"paciente refere sinais e sintomas gripais, com..."


In [14]:
df_limpo.texto_limpo[0]

'anamnese - centro clínico queixa principal: dor inguinoescrotal esq a/e alergias: nega alergia à medicamentos hda: hpp: sinais vitais pa: x fc: fr: t: dor: exame físico: hipótese diagnóstica: hernia inguinal conduta: exames'